In [2]:
import os
import cv2
import moviepy.editor as mp_editor
import math

In [11]:
def standardize_FPS(data, frame_cap):
        # if FPS is lower, duplicate every frame to increase (generate slow video as workaround):        
        if data.shape[0] < frame_cap:
            repeat_for = int(math.ceil(frame_cap / data.shape[0]))
            data = np.repeat(data, repeat_for, axis=0)[:frame_cap]
            # add additional edge case carry out:
            if len(data) < frame_cap:
                extra_needed = frame_cap - len(data)
                extra_array = np.zeros((extra_needed, data.shape[1]), dtype=float)
                data = np.concatenate((data, extra_array))
        return data[:frame_cap]



# increase FPS in video to higher FPS:
def enhance_FPS_to_higher_FPS(video_path, required_fps, save_given_video_copy_as, save_adjusted_video_as):
    
    # read in video of interest:
    mp_clip = mp_editor.VideoFileClip(video_path)
    
    # get width and height of a video:
    width, height = mp_clip.size
    
    # get FPS of current video (MOVIEPY):
    fps = mp_clip.fps
    
    # now, decide, whether we need to simulate frames or not (alternatively, DL can be applied to increase FPS):
    print(f"FPS comparison: current={fps} VS required={required_fps}")
    if fps < required_fps:
        # then we need to find repetition factor for frame duplication:
        repeat_for = int(math.ceil(required_fps / fps))
        print(f"Selected duplication factor = {repeat_for}")
    else:
        print("No need to duplicate. Selection factor set to 0.")
        repeat_for = 0
        
    # setup our video
    original_video_out = cv2.VideoWriter(save_given_video_copy_as, cv2.VideoWriter_fourcc(*'MP4V'), fps, (width, height))
    adjusted_video_out = cv2.VideoWriter(save_adjusted_video_as, cv2.VideoWriter_fourcc(*'MP4V'), required_fps, (width, height))
        
    # start processing video and apply duplication factor as needed:
    video_cap = cv2.VideoCapture(video_path)
    while (video_cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = video_cap.read()
        if ret:
            # write copy to original video:
            original_video_out.write(frame)
            
            # for adjusted sample, we have to make sure we either:
            # do repetition of frames:
            if repeat_for > 0:
                for f in range(repeat_for):
                    cv2.imshow("Frame:", frame)
                    adjusted_video_out.write(frame)
            # or do not do anything:
            else:
                cv2.imshow("Frame:", frame)
                adjusted_video_out.write(frame)
        else:
            print("Failed to read next frame(s). Exiting.")
            break
        # have an option to quit:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
            
    adjusted_video_out.release()
    video_cap.release()
    cv2.destroyAllWindows()
    
    

enhance_FPS_to_higher_FPS(video_path="../../data/original_data/Video_chunks/Video_chunks/trial_truth_016_000.mp4",
                          required_fps=30,
                          save_given_video_copy_as="original_trial_truth_016_000.mp4",
                          save_adjusted_video_as="adjusted_trial_truth_016_000.mp4")

FPS comparison: current=25.0 VS required=30
Selected duplication factor = 2
Failed to read next frame(s). Exiting.


In [12]:
# Convert to GIF:
from PIL import Image
import imageio

def collect_frames(video_path):
    frames = []
    image_count = 0
    video_cap = cv2.VideoCapture(video_path)
    while (video_cap.isOpened()):
        ret, frame = video_cap.read()
        if ret:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(rgb_frame)
        else:
            break
        # have an option to quit:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
            
    print("Images added: ", len(frames))
    return frames
    

def make_gif(video_path, save_gif_as):
    # first read video and collect all available frames:
    video_frames = collect_frames(video_path)
    
    # Now, convert those frames into GIF format:
    print("Saving GIF file")
    with imageio.get_writer(save_gif_as, mode="I") as writer:
        for idx, frame in enumerate(video_frames):
            # print("Adding frame to GIF file: ", idx + 1)
            writer.append_data(frame)
    
    
make_gif(video_path="original_trial_truth_016_000.mp4",
         save_gif_as="original_trial_truth_016_000.gif")

make_gif(video_path="adjusted_trial_truth_016_000.mp4",
         save_gif_as="adjusted_trial_truth_016_000.gif")

Images added:  99
Saving GIF file
Images added:  198
Saving GIF file


In [ ]:
# # OPENCV:
# # get FPS of current video
# vidcap = cv2.VideoCapture(video_path)
# fps = vidcap.get(cv2.CAP_PROP_FPS)
# print(f"OpenCV FPS: {fps}")

# # find video length:
# frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
# duration = frame_count/fps
# print("OpenCV total duration:", duration)

# # find duplication factor: